In [ ]:
from common import *

from matplotlib.ticker import FixedLocator
from matplotlib.lines import Line2D
from datetime import datetime

import geopandas as gp

In [ ]:
ASO_DIR = ASO_DIR / 'Depth-Difference/'
DATES = ['20220421', '20220518']
TIME_DECAY = 'SMRF-2022'
HRRR_MODIS = 'HRRR-MODIS'

In [ ]:
ERW = gp.read_file(str(DATA_DIR / 'Boundaries/Upper_Gunnison.geojson'))
ERW_extent=(ERW.bounds.values.flatten()[0],ERW.bounds.values.flatten()[2], ERW.bounds.values.flatten()[1], ERW.bounds.values.flatten()[3])

In [ ]:
def read_data(path, compressed):
    data = RasterFile(path.as_posix()).band_values()
    
    # Filter difference above 12m and below -12m
    data[data > 12] = np.ma.masked
    data[data < -12] = np.ma.masked
    
    return data[~np.isnan(data)].compressed() if compressed else data

def read_days(compressed=False):
    data = {}
    
    for folder in [TIME_DECAY, HRRR_MODIS]:
        data[folder] = {}
        data[folder][DATES[0]] = read_data(
            ASO_DIR / folder / f'Depth_difference_{DATES[0]}_50m.tif',
            compressed
        )

        data[folder][DATES[1]] = read_data(
            ASO_DIR / folder / f'Depth_difference_{DATES[1]}_50m.tif',
            compressed
        )

    
    return data

## Violin Plot 

In [ ]:
LABELS = ['Time-Decay', 'HRRR-MODIS']

def single_stat(data):
    print(f"     Mean: {np.nanmean(data):.2} m")
    print(f"      Std: {np.nanstd(data):.2} m")
    print(f"   Median: {np.nanmedian(data):.2} m")
    print(f"      Min: {np.nanmin(data):.2} m")
    print(f"      Max: {np.nanmax(data):.2} m")
    print(f" Diff 0.0: {np.where(data == 0.0)[0].size}")
    print(f" Diff %  : {(np.where(data == 0.0)[0].size / data.size):.2%}")
    # print(f"   Diff +/- 0.8: {np.where(np.logical_and(data >= -0.08, data <= 0.08))[0].size}")
    # print(f"       % : {(np.where(np.logical_and(data >= -0.08, data <= 0.08))[0].size / data.size):.2%}")
    print(f"   Pixels: {data.size}")

def print_stats(data):
    for folder in data:
        print(folder)
        for entry in data[folder]:
            print(f"* {entry}")
            single_stat(data[folder][entry])
        print("")

def model_axes(axes, x_pos=1.05):    
    model_cax = axes.twinx()
    model_cax.set_ylabel(r'Model simulation', rotation=270, labelpad=4)
    model_cax.set_yticks([0,1])
    model_cax.set_yticklabels([])
    
    model_cax.annotate("Over",
        xy=(x_pos, 1),
        xytext=(x_pos, .84),
        xycoords='axes fraction',
        va="center", ha="center",
        rotation=270,
        arrowprops=dict(arrowstyle="simple", fc='blue')
    )
    model_cax.annotate("Under",
        xy=(x_pos, 0.00),
        xytext=(x_pos, .16),
        xycoords='axes fraction',
        va="center", ha="center",
        rotation=270,
        arrowprops=dict(arrowstyle="simple", fc='red')
    )


def zero_line(ax):
    for y in [-1, 0, 1]:
        ax.axhline(y, lw=0.8, ls=(1, (1, 3)), color='black', zorder=0, alpha=0.25)

def plot_violin(ax, data, color):
    zero_line(ax)
    
    violin_positions = [0.4, 0.725]
    vp = ax.violinplot(
        data,
        positions=violin_positions,
        widths=0.3,
        showmeans=False,
        showmedians=True,
        showextrema=False,
        quantiles=[
           [0.05, 0.95],
           [0.05, 0.95],
        ],
    )

    color = [color, color]
    ci = 0
    for pc in vp['bodies']:
        pc.set_facecolor(color[ci])
        pc.set_edgecolor('Black')
        pc.set_lw(0.75)
        ci += 1

    for line in ['cquantiles', 'cmedians']:
        vp[line].set_color('black')
        vp[line].set_lw(0.75)

    vp['cmedians'].set_ls(':')
    vp['cmedians'].set_lw(3)
    
        
def plot_differences(ax1, ax2):
    data = read_days(True)
    
    plot_violin(ax1, [data[TIME_DECAY][DATES[0]], data[HRRR_MODIS][DATES[0]]], 'indigo')
    plot_violin(ax2, [data[TIME_DECAY][DATES[1]], data[HRRR_MODIS][DATES[1]]], 'teal')
    
    ax1.set_xticks(
        [0.4, 0.725],
        labels=LABELS
    )
    ax1.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
    ax2.tick_params(top=False, labeltop=False, bottom=False, labelbottom=False)
    ax1.set_xlabel('')
            
    ax2.legend(
        handles=[
            mpatches.Patch(
                facecolor='indigo', ec='grey', alpha=0.5,
                label=datetime.strptime(DATES[0], '%Y%m%d').strftime('%d %b')
            ),
            mpatches.Patch(
                facecolor='teal', ec='grey', alpha=0.5,
                label=datetime.strptime(DATES[1], '%Y%m%d').strftime('%d %b')
            ),
            Line2D([0], [0], color='black', linestyle=':', label='Median'),
            Line2D([0], [0], color='black', label='Quantiles (95%, 5%)')
        ],
        loc='lower right',
        fontsize=8,
        frameon=False,
        bbox_to_anchor=(1.1, -0.18),
        ncol=2
    )
    
    ax1.set_yscale('symlog', linthresh=1)
    ax1.set_ylabel(r'$\Delta$ Snow Depth (m)')
    ax2.set_ylabel(r'$\Delta$ Snow Depth (m)')
    ax1.set_ylim(top=3, bottom=-7)
    ax1.set_yticks([2, 1, 0.5, 0, -0.5, -1, -2, -6])
    ax1.set_yticks(np.arange(-1, 1, 0.1), minor=True)
    ax1.yaxis.set_major_formatter("{x:.1f}")
    ax1.tick_params(axis='y', direction='inout', length=6)

    model_axes(ax1, 1.03)
    model_axes(ax2, 1.03)

In [ ]:
print_stats(read_days(True))

In [ ]:
fig, (ax1, ax2) = plt.subplots(
    nrows=2, sharex=True, sharey=True,
    figsize=(2.75, 8), dpi=300
)
fig.subplots_adjust(hspace=0.025)

plot_differences(ax1, ax2)

## Area Plot

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1 import ImageGrid
from palettable.colorbrewer.diverging import RdBu_5 as RedBlueCmap

RED_BLUE_CMAP = RedBlueCmap.mpl_colormap

In [ ]:
data = read_days()

In [ ]:
def plot_area(data, axes, title='', ylabel=''):
    plt_data = axes.imshow(
        data,
        vmin=-1,
        vmax=1,
        cmap=RED_BLUE_CMAP,
    )
    plt.setp(axes, xticks=[], yticks=[])
    axes.set_facecolor('lightgrey')
    
    axes.set_title(title)
    axes.set_ylabel(ylabel)

    return plt_data

In [ ]:
fig = plt.figure(dpi=300, figsize=(8, 8))

grid = ImageGrid(
    fig, 111,
    nrows_ncols=(2, 2),
    axes_pad=(0.1, 0.1),
    cbar_location="bottom",
    cbar_mode="single",
    cbar_pad="0%",
    cbar_size='3%'
)

plot_area(data[TIME_DECAY][DATES[0]], grid[0], title=LABELS[0], ylabel='21 Apr')
plot_area(data[HRRR_MODIS][DATES[0]], grid[1], title=LABELS[1])
plot_area(data[TIME_DECAY][DATES[1]], grid[2], ylabel='18 May')

cax = grid.cbar_axes[0]
cbar = cax.colorbar(
    plot_area(data[HRRR_MODIS][DATES[1]], grid[3]),
    ticks=np.arange(-1, 1.5, 0.5),
    extend='both'
)
cbar.set_label(
    label=r'$\Delta$ Snow Depth (m)',
)

y_pos = 0.45
cax.annotate("Over",
    xy=(0.88, y_pos),
    xycoords='axes fraction',
    va="center", ha="center",
)
cax.annotate("Under",
    xy=(0.12, y_pos),
    xycoords='axes fraction',
    va="center", ha="center",
)
cax.annotate("Model simulation",
    xy=(0.5, y_pos),
    xycoords='axes fraction',
    va="center", ha="center",
);

## Areas with no snow

In [ ]:
ASO_ACCURACY = 0.00

In [ ]:
def read_data(path, compressed):
    data = RasterFile(path.as_posix()).band_values()
    
    # Filter difference above 12m and below -12m
    data[data > 12] = np.ma.masked
    data[data < -12] = np.ma.masked
    
    return data[~np.isnan(data)].compressed() if compressed else data

def read_days(compressed=False):
    data = {}
    
    for folder in [TIME_DECAY, HRRR_MODIS]:
        data[folder] = {}
        data[folder][DATES[0]] = read_data(
            ASO_DIR / folder / f'ASO_50M_SD_USCOGE_{DATES[0]}_iSnobal_grid.tif',
            compressed
        )

        data[folder][DATES[1]] = read_data(
            ASO_DIR / folder / f'ASO_50M_SD_USCOGE_{DATES[1]}_iSnobal_grid.tif',
            compressed
        )

    
    return data

In [ ]:
aso_data = read_days()

### Mask NaN and areas with snow 

In [ ]:
no_snow_mask = ~(np.isnan(aso_data['HRRR-MODIS']['20220421']).mask | (aso_data['HRRR-MODIS']['20220421'] <= ASO_ACCURACY).data)

### April

In [ ]:
no_snow_aso = np.ma.masked_where(no_snow_mask, aso_data['HRRR-MODIS']['20220421'])
no_snow_SMRF_apr = np.ma.masked_where(no_snow_mask, data['SMRF-2022']['20220421'])
no_snow_HRRR_apr = np.ma.masked_where(no_snow_mask, data['HRRR-MODIS']['20220421'])

### May

In [ ]:
no_snow_mask = ~(np.isnan(aso_data['HRRR-MODIS']['20220518']).mask | (aso_data['HRRR-MODIS']['20220518'] <= ASO_ACCURACY).data)

In [ ]:
no_snow_aso = np.ma.masked_where(no_snow_mask, aso_data['HRRR-MODIS']['20220518'])
no_snow_SMRF_may = np.ma.masked_where(no_snow_mask, data['SMRF-2022']['20220518'])
no_snow_HRRR_may = np.ma.masked_where(no_snow_mask, data['HRRR-MODIS']['20220518'])

In [ ]:
from palettable.colorbrewer.sequential import Blues_9 as BlueCmap

BLUE_CMAP = BlueCmap.mpl_colormap

In [ ]:
def plot_area(data, axes, title='', ylabel=''):
    plt_data = axes.imshow(
        data,
        vmin=0,
        vmax=0.5,
        cmap=BLUE_CMAP,
        extent=ERW_extent,
    )
    plt.setp(axes, xticks=[], yticks=[])
    axes.set_facecolor('lightgrey')
    
    axes.set_title(title)
    axes.set_ylabel(ylabel)

    return plt_data

In [ ]:
fig = plt.figure(dpi=300, figsize=(8, 8))

grid = ImageGrid(
    fig, 111,
    nrows_ncols=(2, 2),
    axes_pad=(0.1, 0.1),
    cbar_location="bottom",
    cbar_mode="single",
    cbar_pad="0%",
    cbar_size='3%'
)

plot_area(no_snow_SMRF_apr, grid[0], title=LABELS[0], ylabel='21 Apr')
plot_area(no_snow_HRRR_apr, grid[1], title=LABELS[1])
plot_area(no_snow_SMRF_may, grid[2], ylabel='18 May')

[ERW.boundary.plot(color='grey', ax=el, lw=0.5) for el in grid]

cax = grid.cbar_axes[0]
cbar = cax.colorbar(
    plot_area(no_snow_HRRR_may, grid[3]),
    ticks=np.arange(0, 1, 0.1),
    extend='max'
)
cbar.set_label(
    label=r'Snow Depth (m)',
)

In [ ]:
def percent_below_threshold(data, threshold):
    data = data.compressed()
    print(f"Percent: {data[np.where(data <= threshold)].size / data.size:.2%}")
    print(f"Mean: {np.nanmean(data):.3} m")

In [ ]:
percent_below_threshold(no_snow_SMRF_apr, ASO_ACCURACY)

In [ ]:
percent_below_threshold(no_snow_HRRR_apr, ASO_ACCURACY)

In [ ]:
percent_below_threshold(no_snow_SMRF_may, ASO_ACCURACY)

In [ ]:
percent_below_threshold(no_snow_HRRR_may, ASO_ACCURACY)

In [ ]:
def count_y(data):
    return np.flip(np.ma.masked_where(data <= ASO_ACCURACY, data).count(axis=1))

In [ ]:
fig, (ax1, ax2) = plt.subplots(
    nrows=2, sharex=True, sharey=True,
    dpi=300, figsize=(2.5,7.65)
)
fig.subplots_adjust(hspace=0.034)

# ax1.plot(count_y(no_snow_SMRF_apr), np.arange(no_snow_SMRF_apr.shape[0]), color='indigo', lw=.5, alpha=0.5)
ax1.fill_betweenx(np.arange(no_snow_SMRF_apr.shape[0]), 0, count_y(no_snow_SMRF_apr), color='indigo', alpha=0.2, label='Time-Decay')
ax1.plot(count_y(no_snow_HRRR_apr), np.arange(no_snow_HRRR_apr.shape[0]), label='HRRR-MODIS', color='black', lw=0.75)

# ax2.plot(count_y(no_snow_SMRF_may), np.arange(no_snow_SMRF_may.shape[0]), color='teal', lw=.5, alpha=0.5)
ax2.fill_betweenx(np.arange(no_snow_SMRF_may.shape[0]), 0, count_y(no_snow_SMRF_may), label='Time-Decay', color='teal', alpha=0.2)
ax2.plot(count_y(no_snow_HRRR_may), np.arange(no_snow_HRRR_may.shape[0]), label='HRRR-MODIS', color='black', lw=0.75)

ax1.set_title('Latitude Difference')

ax2.set_xticks(np.arange(0, 130, 25))
ax2.tick_params(axis='x', which='major', labelsize=10)
ax2.set_xlabel('Grid Cell Count')
ax2.set_xlim(left=-0.5)

ax2.set_ylim(0, no_snow_SMRF_apr.shape[0])
ax2.set_yticks([])
ax2.set_yticklabels([])

ax1.legend(
    loc='upper right',
    fontsize=8,
    frameon=False,
)
ax2.legend(
    loc='upper right',
    fontsize=8,
    frameon=False,
);